# GPS Fitness Analysis

In [2]:
import matplotlib as plt
import numpy as np
import pandas as pd
import gpxpy
import datetime
from geopy import distance
from math import sqrt, floor
import numpy as np
import pandas as pd
#import plotly.plotly as py
import plotly.graph_objs as go
import haversine

In [94]:
boulder_gpx = open("activity_5070523877.gpx", 'r')
boulder_location = gpxpy.parse(boulder_gpx)
boulder_location

GPX(tracks=[GPXTrack(name='Bear Peak and South Boulder Peak', segments=[GPXTrackSegment(points=[...])])])

In [95]:
len(boulder_location.tracks[0].segments[0].points)

6406

In [96]:
boulder_location.tracks[0].segments[0].points

[GPXTrackPoint(39.940845435485244, -105.2601823490113, elevation=1732.5999755859375, time=datetime.datetime(2020, 6, 7, 12, 11, 16, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.94085364975035, -105.26017782278359, elevation=1744.5999755859375, time=datetime.datetime(2020, 6, 7, 12, 11, 17, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.9408651329577, -105.26017463766038, elevation=1750.199951171875, time=datetime.datetime(2020, 6, 7, 12, 11, 18, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.940883070230484, -105.26017162017524, elevation=1753.5999755859375, time=datetime.datetime(2020, 6, 7, 12, 11, 19, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.94090486317873, -105.26016910560429, elevation=1755.0, time=datetime.datetime(2020, 6, 7, 12, 11, 20, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.94094886817038, -105.26017145253718, elevation=1755.800048828125, time=datetime.datetime(2020, 6, 7, 12, 11, 22, tzinfo=SimpleTZ("Z"))),
 GPXTrackPoint(39.94097066111863, -105.26017379947007, elevation=1756.0, time=

In [92]:
boulder_location = boulder_location.tracks[0].segments[0].points

In [20]:
## Start Position
start = boulder_location[0]
## End Position
finish = boulder_location[-1]

In [159]:
boulder_df = pd.DataFrame(columns=['lon', 'lat', 'ele', 'time', 'hr', 'temp'])
for point in boulder_data:
    boulder_df = boulder_df.append({'lon': point.longitude, 'lat' : point.latitude, 'ele' : point.elevation*3.28, 'time' : point.time}, ignore_index=True)

In [160]:
boulder_df

,lon,lat,ele,time,hr,temp
0,-105.260182,39.940845,5682.92792,2020-06-07 12:11:16+00:00,NaN,NaN
1,-105.260178,39.940854,5722.28792,2020-06-07 12:11:17+00:00,NaN,NaN
2,-105.260175,39.940865,5740.65584,2020-06-07 12:11:18+00:00,NaN,NaN
3,-105.260172,39.940883,5751.80792,2020-06-07 12:11:19+00:00,NaN,NaN
4,-105.260169,39.940905,5756.40000,2020-06-07 12:11:20+00:00,NaN,NaN
...,...,...,...,...,...,...
6401,-105.258004,39.938618,5772.80000,2020-06-07 16:36:35+00:00,NaN,NaN
6402,-105.257999,39.938625,5773.45584,2020-06-07 16:36:39+00:00,NaN,NaN
6403,-105.257996,39.938627,5773.45584,2020-06-07 16:36:40+00:00,NaN,NaN
6404,-105.257985,39.938635,5775.42416,2020-06-07 16:36:42+00:00,NaN,NaN


In [34]:
max(boulder_df.alt)*3.28

8495.855839843749

In [161]:
boulder_df['hr'] = hr

In [162]:
boulder_df

,lon,lat,ele,time,hr,temp
0,-105.260182,39.940845,5682.92792,2020-06-07 12:11:16+00:00,131,NaN
1,-105.260178,39.940854,5722.28792,2020-06-07 12:11:17+00:00,130,NaN
2,-105.260175,39.940865,5740.65584,2020-06-07 12:11:18+00:00,131,NaN
3,-105.260172,39.940883,5751.80792,2020-06-07 12:11:19+00:00,131,NaN
4,-105.260169,39.940905,5756.40000,2020-06-07 12:11:20+00:00,131,NaN
...,...,...,...,...,...,...
6401,-105.258004,39.938618,5772.80000,2020-06-07 16:36:35+00:00,107,NaN
6402,-105.257999,39.938625,5773.45584,2020-06-07 16:36:39+00:00,108,NaN
6403,-105.257996,39.938627,5773.45584,2020-06-07 16:36:40+00:00,114,NaN
6404,-105.257985,39.938635,5775.42416,2020-06-07 16:36:42+00:00,119,NaN


# Method RE: Jeffrey James

In [231]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from collections import Counter, defaultdict
import datetime
import xml.etree.ElementTree as ET

In [172]:
with open("activity_5070523877.gpx") as fp:
    soup = BeautifulSoup(fp, "html.parser")

In [227]:
soup

<?xml version="1.0" encoding="UTF-8"?>
<gpx creator="Garmin Connect" version="1.1" xmlns="http://www.topografix.com/GPX/1/1" xmlns:ns2="http://www.garmin.com/xmlschemas/GpxExtensions/v3" xmlns:ns3="http://www.garmin.com/xmlschemas/TrackPointExtension/v1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemalocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/11.xsd">
<metadata>
<link href="connect.garmin.com"/>
<text>Garmin Connect</text>
<time>2020-06-07T12:11:16.000Z</time>
</metadata>
<trk>
<name>Bear Peak and South Boulder Peak</name>
<type>hiking</type>
<trkseg>
<trkpt lat="39.94084543548524379730224609375" lon="-105.26018234901130199432373046875">
<ele>1732.5999755859375</ele>
<time>2020-06-07T12:11:16.000Z</time>
<extensions>
<ns3:trackpointextension>
<ns3:atemp>23.0</ns3:atemp>
<ns3:hr>131</ns3:hr>
</ns3:trackpointextension>
</extensions>
</trkpt>
<trkpt lat="39.94085364975035190582275390625" lon="-105.26017782278358936309814453125">
<ele>1744.59

In [225]:
soup.trk.find_all("type")[0].get_text()

'hiking'

In [226]:
soup.trk.find_all("name")[0].get_text()

'Bear Peak and South Boulder Peak'

In [173]:
points = soup.find_all("trkpt")

In [174]:
len(points)

6406

## Unpacking Latitude and Longitude

In [254]:
dir(points[0])

['__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_find_all',
 '_find_one',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'children',
 'clear',
 'contents',
 'decode',
 'decode_contents',
 'decompose',
 'decomposed',
 'descendants',
 'encode',
 'encode_contents',
 'extend',
 'extract',
 'fetchNextSiblings',
 'fetchParents',
 'fetchPrevious',
 'fetchPreviousSiblings',
 '

In [250]:
points[0]

<trkpt lat="39.94084543548524379730224609375" lon="-105.26018234901130199432373046875">
<ele>1732.5999755859375</ele>
<time>2020-06-07T12:11:16.000Z</time>
<extensions>
<ns3:trackpointextension>
<ns3:atemp>23.0</ns3:atemp>
<ns3:hr>131</ns3:hr>
</ns3:trackpointextension>
</extensions>
</trkpt>

In [255]:
points[0].attrs

{'lat': '39.94084543548524379730224609375',
 'lon': '-105.26018234901130199432373046875'}

In [266]:
float(points[0].attrs['lat'])

39.940845435485244

In [260]:
points[0].ele.get_text()

str

In [190]:
points[0].extensions.findChildren()[2].get_text()

'131'

In [267]:
lat = []
lon = []
ele = []
time = []

hr = []
atemp = []

for p in points:
    lat.append(float(p.attrs['lat']))
    hr.append(float([item.get_text() for item in p.findChildren()][5]))
    atemp.append(float([item.get_text() for item in p.findChildren()][4]))

In [268]:
print(len(lat))
print(len(hr))
print(len(atemp))

6406
6406
6406


In [277]:
print(lat[1])
type(lat[1])

39.94085364975035
<class 'float'>


In [170]:
boulder_df = pd.DataFrame(columns=['lon', 'lat', 'ele', 'time', 'hr', 'temp'])
boulder_df.lon = points.lat